In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

import chardet

file_path = '/content/drive/MyDrive/Thesis/posts1352.csv'

# Detect the file encoding
with open(file_path, 'rb') as f:
    result = chardet.detect(f.read())

encoding = result['encoding']

# Use the detected encoding to read the file
df1 = pd.read_csv(file_path, encoding=encoding)

In [13]:
stopword_extra = ["always","even","still","ever","also","already","yet","basically","actually","please","ago","probable","probably","however","instead","quite","nt","na","u","gon","lol","im","ca","us","cnt","wo","em","etc","ll","r"]
#"cant","shouldnt","wont","lah","dont","never","aint"]

In [5]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
df1

,No,Category,SubCategory,Post,HateSpeech
0,1,Race,Chinese,Chinese are pigs!,1
1,2,Race,Indian,I don't like pajeets they smell like curry,1
2,3,NaN,NaN,Israel is defending itself from Hamas,0
3,4,Race,Indian,"Indians always smell stinky and weird, what is...",1
4,5,Gender,Female,Women are so emotional they can't handle anyth...,1
...,...,...,...,...,...
1347,1348,Gender,Female,Malaysia's female labour force participation r...,0
1348,1349,Religion,Buddhism,You cannot build a temple in an area with majo...,1
1349,1350,Religion,Buddhism,"Muslims should not enter temple, or they might...",1
1350,1351,Gender,Male,Men that don't have job are useless piece of s...,1


In [3]:
df2 = df1.dropna()

In [6]:
df2["Post"] = df2["Post"].apply(lambda x: sent_tokenize(x)) #this only "tokenize" post with multuple lines into sentences

<ipython-input-6-f7deb7e1d76e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["Post"] = df2["Post"].apply(lambda x: sent_tokenize(x)) #this only "tokenize" post with multuple lines into sentences


In [7]:
df2

,No,Category,SubCategory,Post,HateSpeech
0,1,Race,Chinese,[Chinese are pigs!],1
1,2,Race,Indian,[I don't like pajeets they smell like curry],1
3,4,Race,Indian,"[Indians always smell stinky and weird, what i...",1
4,5,Gender,Female,[Women are so emotional they can't handle anyt...,1
5,6,Race,Malay,[Malays are dependant on subsidiaries from the...,1
...,...,...,...,...,...
1347,1348,Gender,Female,[Malaysia's female labour force participation ...,0
1348,1349,Religion,Buddhism,[You cannot build a temple in an area with maj...,1
1349,1350,Religion,Buddhism,"[Muslims should not enter temple, or they migh...",1
1350,1351,Gender,Male,[Men that don't have job are useless piece of ...,1


In [8]:
df2 = df2.explode('Post')

df2 = df2.reset_index(drop=True)

In [9]:
df2

,No,Category,SubCategory,Post,HateSpeech
0,1,Race,Chinese,Chinese are pigs!,1
1,2,Race,Indian,I don't like pajeets they smell like curry,1
2,4,Race,Indian,"Indians always smell stinky and weird, what is...",1
3,5,Gender,Female,Women are so emotional they can't handle anyth...,1
4,6,Race,Malay,Malays are dependant on subsidiaries from the ...,1
...,...,...,...,...,...
1634,1349,Religion,Buddhism,You cannot build a temple in an area with majo...,1
1635,1350,Religion,Buddhism,"Muslims should not enter temple, or they might...",1
1636,1351,Gender,Male,Men that don't have job are useless piece of s...,1
1637,1352,Gender,Female,Women are dick machines.,1


In [14]:
import string
import re
from textblob import TextBlob
from nltk.corpus import wordnet
from nltk.metrics import edit_distance
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize

import spacy
nlp = spacy.load('en_core_web_sm') #, disable=['parser', 'ner'])

stopwords = nltk.corpus.stopwords.words('english')
stopwords += stopword_extra

negative_list = ['not','never','ain','aint','no','neither','nor','nt','cant','dont',"cnt",'wont',"shouldnt"]

def preprocess(text):

    #Generating the list of words in a sentence (hastags and other punctuations removed)
    text_blob = TextBlob(text)
    text = ' '.join(text_blob.words)

    # remove number
    text = re.sub(r'[0-9]', '', text)

    # lowercase
    text = text.lower()

    text = text.replace('/',' ')

    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')


    text = word_tokenize(text)

    #keep tokens that are alphabet characters
    text = [t for t in text if t.isalpha()]

    # replace the negation token
    replacer  = AntonymReplacer()
    text = replacer.replace_negations(text)

    # remove the stopwords
    text = [i for i in text if i not in stopwords]

    #remove empty token
    text = [t for t in text if len(t) > 0]

    return text


class AntonymReplacer(object):
    def replace(self, word, pos=None):
        antonyms = set()

        for syn in wordnet.synsets(word, pos=pos):
            for lemma in syn.lemmas():
                for antonym in lemma.antonyms():
                    antonyms.add(antonym.name())

        if len(antonyms) >= 1:
            return antonyms.pop()
        else:
            return None

    def replace_negations(self, sent):
        i, l = 0, len(sent)
        words = []

        while i < l:
            word = sent[i]

            if word in negative_list and i+1 < l:
                ant = self.replace(sent[i+1])

                if ant:
                    words.append(ant)
                    i += 2
                    continue

            words.append(word)
            i += 1

        return words

In [15]:
#Preprocess the input data
cleaned_text = []
counter = 1

for text in df2["Post"]:
    if counter % 100 == 0:
        print(f'Preprocessing row {counter}')
    counter+=1

    cleaned_text.append(preprocess(text))

Preprocessing row 100
Preprocessing row 200
Preprocessing row 300
Preprocessing row 400
Preprocessing row 500
Preprocessing row 600
Preprocessing row 700
Preprocessing row 800
Preprocessing row 900
Preprocessing row 1000
Preprocessing row 1100
Preprocessing row 1200
Preprocessing row 1300
Preprocessing row 1400
Preprocessing row 1500
Preprocessing row 1600


In [16]:
df2["PreprocessedToken"] = cleaned_text

In [17]:
for i in range(len(df2)):

    if len(df2["PreprocessedToken"][i]) == 0:
        df2 = df2.drop(i)

In [ ]:
df2.to_csv("/content/drive/MyDrive/Thesis/posts1352_standard.csv",index=False)

In [19]:
df2

,No,Category,SubCategory,Post,HateSpeech,PreprocessedToken
0,1,Race,Chinese,Chinese are pigs!,1,"[chinese, pigs]"
1,2,Race,Indian,I don't like pajeets they smell like curry,1,"[unlike, pajeets, smell, like, curry]"
2,4,Race,Indian,"Indians always smell stinky and weird, what is...",1,"[indians, smell, stinky, weird]"
3,5,Gender,Female,Women are so emotional they can't handle anyth...,1,"[women, emotional, handle, anything, keep, cry..."
4,6,Race,Malay,Malays are dependant on subsidiaries from the ...,1,"[malays, dependant, subsidiaries, government]"
...,...,...,...,...,...,...
1634,1349,Religion,Buddhism,You cannot build a temple in an area with majo...,1,"[build, temple, area, majority, muslims]"
1635,1350,Religion,Buddhism,"Muslims should not enter temple, or they might...",1,"[muslims, exit, temple, might, accidentaly, co..."
1636,1351,Gender,Male,Men that don't have job are useless piece of s...,1,"[men, abstain, job, useless, piece, sheet]"
1637,1352,Gender,Female,Women are dick machines.,1,"[women, dick, machines]"


Making a similar one without removing the None category

In [ ]:
df1["Post"] = df1["Post"].apply(lambda x: sent_tokenize(x)) #this only "tokenize" post with multuple lines into sentences

In [ ]:
df1 = df1.explode('Post')

df1 = df1.reset_index(drop=True)

In [ ]:
#Preprocess the input data
cleaned_text2 = []
counter = 1

for text in df1["Post"]:
    if counter % 100 == 0:
        print(f'Preprocessing row {counter}')
    counter+=1
    cleaned_text2.append(preprocess(text))

Preprocessing row 100
Preprocessing row 200
Preprocessing row 300
Preprocessing row 400
Preprocessing row 500
Preprocessing row 600
Preprocessing row 700
Preprocessing row 800
Preprocessing row 900
Preprocessing row 1000
Preprocessing row 1100
Preprocessing row 1200
Preprocessing row 1300
Preprocessing row 1400
Preprocessing row 1500
Preprocessing row 1600
Preprocessing row 1700
Preprocessing row 1800
Preprocessing row 1900


In [ ]:
df1["PreprocessedToken"] = cleaned_text2

In [ ]:
df1.to_csv('/content/drive/MyDrive/Thesis/posts1352_complete.csv',index=False)

In [ ]:
df1

,No,Category,SubCategory,Post,HateSpeech,PreprocessedToken
0,1,Race,Chinese,Chinese are pigs!,1,"[chinese, pigs]"
1,2,Race,Indian,I don't like pajeets they smell like curry,1,"[unalike, pajeets, smell, like, curry]"
2,3,NaN,NaN,Israel is defending itself from Hamas,0,"[israel, defending, hamas]"
3,4,Race,Indian,"Indians always smell stinky and weird, what is...",1,"[indians, smell, stinky, weird]"
4,5,Gender,Female,Women are so emotional they can't handle anyth...,1,"[women, emotional, handle, anything, keep, cry..."
...,...,...,...,...,...,...
1972,1349,Religion,Buddhism,You cannot build a temple in an area with majo...,1,"[build, temple, area, majority, muslims]"
1973,1350,Religion,Buddhism,"Muslims should not enter temple, or they might...",1,"[muslims, exit, temple, might, accidentaly, co..."
1974,1351,Gender,Male,Men that don't have job are useless piece of s...,1,"[men, lack, job, useless, piece, sheet]"
1975,1352,Gender,Female,Women are dick machines.,1,"[women, dick, machines]"


In [23]:
df2['HateSpeech'].value_counts(normalize=True, dropna=False)

HateSpeech
1    0.501852
0    0.498148
Name: proportion, dtype: float64

In [26]:
race = df2[df2['Category']=='Race']
religion = df2[df2['Category']=='Religion']
gender = df2[df2['Category']=='Gender']

In [27]:
race['HateSpeech'].value_counts(normalize=True, dropna=False)

HateSpeech
1    0.502098
0    0.497902
Name: proportion, dtype: float64

In [25]:
religion['HateSpeech'].value_counts(normalize=True, dropna=False)

HateSpeech
1    0.500838
0    0.499162
Name: proportion, dtype: float64

In [28]:
gender['HateSpeech'].value_counts(normalize=True, dropna=False)

HateSpeech
1    0.503247
0    0.496753
Name: proportion, dtype: float64